In [2]:
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery
import numpy as np
import re

In [3]:
os.chdir('G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Dados para subir\\Tabelas finais')


In [4]:
df = pd.read_csv('SIAPE_ativos_regulatoria.csv')
df

,ano,ministerios,servidores_ativos_ministerios,total_servidores_ativos,perc_servidores
0,1999,ministerio da fazenda,27990,482854,5.80
1,1999,ministerio da justica,23450,482854,4.86
2,1999,ministerio do planejamento desenvolvimento e g...,36797,482854,7.62
3,2000,ministerio da fazenda,26184,478435,5.47
4,2000,ministerio da justica,22644,478435,4.73
...,...,...,...,...,...
57,2017,ministerio do planejamento desenvolvimento e g...,32567,614925,5.30
58,2018,ministerio da fazenda,30967,611916,5.06
59,2018,ministerio da justica,31642,611916,5.17
60,2018,ministerio da transparencia e cgu,2308,611916,0.38


In [5]:
df['ministerios'].unique()


array(['ministerio da fazenda', 'ministerio da justica',
       'ministerio do planejamento desenvolvimento e gestao',
       'ministerio da transparencia e cgu'], dtype=object)

In [6]:
df['ministerios'] = df['ministerios'].str.title()

In [7]:
df

,ano,ministerios,servidores_ativos_ministerios,total_servidores_ativos,perc_servidores
0,1999,Ministerio Da Fazenda,27990,482854,5.80
1,1999,Ministerio Da Justica,23450,482854,4.86
2,1999,Ministerio Do Planejamento Desenvolvimento E G...,36797,482854,7.62
3,2000,Ministerio Da Fazenda,26184,478435,5.47
4,2000,Ministerio Da Justica,22644,478435,4.73
...,...,...,...,...,...
57,2017,Ministerio Do Planejamento Desenvolvimento E G...,32567,614925,5.30
58,2018,Ministerio Da Fazenda,30967,611916,5.06
59,2018,Ministerio Da Justica,31642,611916,5.17
60,2018,Ministerio Da Transparencia E Cgu,2308,611916,0.38


In [8]:
def transformar(nome):
    nome = re.sub(r"\bMinisterio\b", "Ministério",nome)
    nome = re.sub(r"\bCgu\b", "CGU",nome)
    nome = re.sub(r"\bGestao\b", "Gestão",nome)
    nome = re.sub(r"\bJustica\b", "Justiça",nome)
    nome = re.sub(r"\bTransparencia\b", "Transparência",nome)
    return nome

In [9]:
df['ministerios']= df['ministerios'].apply(transformar)
df

,ano,ministerios,servidores_ativos_ministerios,total_servidores_ativos,perc_servidores
0,1999,Ministério Da Fazenda,27990,482854,5.80
1,1999,Ministério Da Justiça,23450,482854,4.86
2,1999,Ministério Do Planejamento Desenvolvimento E G...,36797,482854,7.62
3,2000,Ministério Da Fazenda,26184,478435,5.47
4,2000,Ministério Da Justiça,22644,478435,4.73
...,...,...,...,...,...
57,2017,Ministério Do Planejamento Desenvolvimento E G...,32567,614925,5.30
58,2018,Ministério Da Fazenda,30967,611916,5.06
59,2018,Ministério Da Justiça,31642,611916,5.17
60,2018,Ministério Da Transparência E CGU,2308,611916,0.38


In [10]:
df['ministerios'].unique()


array(['Ministério Da Fazenda', 'Ministério Da Justiça',
       'Ministério Do Planejamento Desenvolvimento E Gestão',
       'Ministério Da Transparência E CGU'], dtype=object)

In [11]:
df.rename(columns={'perc_servidores' : 'per_servidores'}, inplace=True) #padronizando


In [12]:
df['per_servidores'] = df['per_servidores']/100 


In [15]:
df = df.drop(['total_servidores_ativos'], axis= 1)

In [16]:
df

,ano,ministerios,servidores_ativos_ministerios,per_servidores
0,1999,Ministério Da Fazenda,27990,0.0580
1,1999,Ministério Da Justiça,23450,0.0486
2,1999,Ministério Do Planejamento Desenvolvimento E G...,36797,0.0762
3,2000,Ministério Da Fazenda,26184,0.0547
4,2000,Ministério Da Justiça,22644,0.0473
...,...,...,...,...
57,2017,Ministério Do Planejamento Desenvolvimento E G...,32567,0.0530
58,2018,Ministério Da Fazenda,30967,0.0506
59,2018,Ministério Da Justiça,31642,0.0517
60,2018,Ministério Da Transparência E CGU,2308,0.0038


Subindo para o BQ

In [17]:
client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao')

c:\Users\ana.sales_republica\Anaconda\lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [18]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência da observação'),
 bigquery.SchemaField('ministerios','STRING',description='Ministérios observados'),
 bigquery.SchemaField('servidores_ativos_ministerios', 'INTEGER',description='Número de servidores ativos por ministério'),
 bigquery.SchemaField('per_servidores','FLOAT', description='Percentual de servidores ativos por ministério ')]


In [19]:
table_ref = dataset_ref.table('SIAPE_ativos_regulatoria') 
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()  

LoadJob<project=repositoriodedadosgpsp, location=US, id=3cb11f67-0a88-4599-8063-fefefe46ae1c>